In [1]:
# Load Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.ticker as mtick
import sqlite3
import seaborn as sns
from matplotlib.pyplot import figure
from bs4 import BeautifulSoup
import time
import requests     # to get images
import shutil       # to save files locally
import datetime
from scipy.stats import norm
import warnings
warnings.filterwarnings('ignore')
import requests
import json
#import xgboost
#from xgboost import XGBClassifier
from random import randint
import  random
import os
os.chdir('C:/Users/tmcro/OneDrive/Data Science/Personal_Projects/Sports/UFC_Prediction')
from cmath import nan
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
# scrape UFCStats website for next event
def get_next_event_ufcstats():
    url = 'http://www.ufcstats.com/statistics/events/upcoming'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    # get events
    event1 = soup.find('td', class_='b-statistics__table-col')
    event1_txt = soup.find('td', class_='b-statistics__table-col').text
    event_txt = event1_txt.replace('   ', '').replace('\n', '').strip()
    event_title = event_txt.split('  ')[0]
    event_date = event_txt.split('  ')[1]
    event1_url = event1.find('a')['href']
    data = pd.DataFrame({'event_title': [event_title], 'event_url': [event1_url], 'event_date': [event_date]})


    return data

In [3]:
next_event = get_next_event_ufcstats()
next_event

,event_title,event_url,event_date
0,UFC Fight Night: Rodriguez vs. Lemos,http://www.ufcstats.com/event-details/756f4590...,"November 05, 2022"


### Grab all data from all fighters fighting in next event

In [4]:
def get_fighter_urls(event_details_url):
    page = requests.get(event_details_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    # get events
    events = soup.find_all('tr', class_='b-fight-details__table-row b-fight-details__table-row__hover js-fight-details-click')
    n = 0
    next_event_data = pd.DataFrame()

    for event in events:
        fighters = events[n].find_all('p', class_='b-fight-details__table-text')
        fighter1 = fighters[0].text
        fighter1 = fighter1.replace('  ', '').replace('\n', '').strip()
        fighter2 = fighters[1].text
        fighter2 = fighter2.replace('  ', '').replace('\n', '').strip()
        fighter1_url = fighters[0].find('a')['href']
        fighter2_url = fighters[1].find('a')['href']
        next_event_data = next_event_data.append({'fighter1' :fighter1, 'fighter2:' : fighter2, 'fighter1_url': fighter1_url, 'fighter2_url':fighter2_url, 'fight#' : n+1}, ignore_index = True)
        n += 1

    return next_event_data


In [5]:
fighter_urls = get_fighter_urls(next_event['event_url'].values[0])
fighter_urls

,fighter1,fighter2:,fighter1_url,fighter2_url,fight#
0,Marina Rodriguez,Amanda Lemos,http://www.ufcstats.com/fighter-details/cd2c4d...,http://www.ufcstats.com/fighter-details/3df549...,1
1,Neil Magny,Daniel Rodriguez,http://www.ufcstats.com/fighter-details/84b3e7...,http://www.ufcstats.com/fighter-details/8a1f3b...,2
2,Chase Sherman,Josh Parisian,http://www.ufcstats.com/fighter-details/01b352...,http://www.ufcstats.com/fighter-details/b5da8f...,3
3,Tagir Ulanbekov,Nate Maness,http://www.ufcstats.com/fighter-details/f00ac0...,http://www.ufcstats.com/fighter-details/6e9f40...,4
4,Grant Dawson,Mark Madsen,http://www.ufcstats.com/fighter-details/99bd51...,http://www.ufcstats.com/fighter-details/da0995...,5
5,Jailton Almeida,Maxim Grishin,http://www.ufcstats.com/fighter-details/41e83a...,http://www.ufcstats.com/fighter-details/950d0e...,6
6,Darrick Minner,Shayilan Nuerdanbieke,http://www.ufcstats.com/fighter-details/34e516...,http://www.ufcstats.com/fighter-details/0a73ac...,7
7,Miranda Maverick,Shanna Young,http://www.ufcstats.com/fighter-details/f29a63...,http://www.ufcstats.com/fighter-details/568bba...,8
8,Mario Bautista,Benito Lopez,http://www.ufcstats.com/fighter-details/bc711b...,http://www.ufcstats.com/fighter-details/773f7f...,9
9,Polyana Viana,Jinh Yu Frey,http://www.ufcstats.com/fighter-details/9673a4...,http://www.ufcstats.com/fighter-details/d05cb1...,10


In [6]:
def get_fighter_previous_fights(fighter_details_url):
    page = requests.get(fighter_details_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    # get fighter name from top of fighter details page
    fighter_name = soup.find('span', class_='b-content__title-highlight').text
    fighter_name = fighter_name.replace('\n', '').replace('  ', '').strip()

    #get fighter last fights data
    fighter_last_fights = soup.find_all('tr', class_='b-fight-details__table-row b-fight-details__table-row__hover js-fight-details-click')
    n = 0
    fighter_last_fights_data = pd.DataFrame()

    for fight in fighter_last_fights:
        urls = fighter_last_fights[n].find_all('a')
        fight_url = urls[0]['href']
        fighter_last_fights_data = fighter_last_fights_data.append({'fight_url' :fight_url}, ignore_index = True)
        n += 1

    return fighter_last_fights_data


In [7]:
fighter_prevFights_urls = get_fighter_previous_fights(fighter_urls['fighter1_url'].values[0])
fighter_prevFights_urls

,fight_url
0,http://www.ufcstats.com/fight-details/9321b3c9...
1,http://www.ufcstats.com/fight-details/149d6b90...
2,http://www.ufcstats.com/fight-details/b948102c...
3,http://www.ufcstats.com/fight-details/cd08b68f...
4,http://www.ufcstats.com/fight-details/5632f236...
5,http://www.ufcstats.com/fight-details/3b8cac99...
6,http://www.ufcstats.com/fight-details/d93b589b...
7,http://www.ufcstats.com/fight-details/f5a038c1...
8,http://www.ufcstats.com/fight-details/a7492cbf...
9,http://www.ufcstats.com/fight-details/c545843a...


In [8]:
fighter_previous_fight_urls = list(fighter_prevFights_urls['fight_url'].values)

# Streamlit Functionality

We also need it to work in streamlit, without taking forever. So, there should be an updated fights_df with the NEXT fights added, which can then update the calculations on a consistant basis. 

In [9]:
# make a looping version of get_next_events(url) 
def get_next_events_ufc_com(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    # get events
    events = soup.find_all('div', class_='c-card-event--result__info')
    n = 0
    next_events_data = pd.DataFrame()

    for event in events:
        event_txt = events[n].text
        event_url = events[n].find('a')['href']
        event_url = 'https://www.ufc.com' + event_url
        event_title = event_txt.split('\n')[1]
        event_time = event_txt.split('/')[1]
        next_events_data = next_events_data.append({'event_title': event_title, 'event_url': event_url, 'event_date': event_time}, ignore_index=True)
        n += 1

    # only keep top 6 events
    next_events_data = next_events_data.head(6)
    return next_events_data

In [10]:
nel = get_next_events_ufc_com('https://www.ufc.com/events')
nel

,event_title,event_url,event_date
0,Rodriguez vs Lemos,https://www.ufc.com/event/ufc-fight-night-nove...,7:00 PM EDT
1,Adesanya vs Pereira,https://www.ufc.com/event/ufc-281,10:00 PM EST
2,Lewis vs Spivac,https://www.ufc.com/event/ufc-fight-night-nove...,4:00 PM EST
3,Thompson vs Holland,https://www.ufc.com/event/ufc-fight-night-dece...,10:00 PM EST
4,Prochazka vs Teixeira,https://www.ufc.com/event/ufc-282,10:00 PM EST
5,Cannonier vs Strickland,https://www.ufc.com/event/ufc-fight-night-dece...,7:00 PM EST


### Get next events from ufcstats

In [11]:
url = 'http://www.ufcstats.com/statistics/events/upcoming'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
    # get events
event = soup.findAll('i', class_ = 'b-statistics__table-content')
urls = soup.findAll('a', class_ = 'b-link b-link_style_black')
dates = soup.findAll('span', class_ = 'b-statistics__date')
n = 0
next_event_data = pd.DataFrame()

for i in event:
    # get event title, date, and url
    title = event[n].text
    title = title.replace('\n', '').strip()
    ti = title.split('  ')[0]
    date = dates[n].text.replace('\n', '').strip()
    #date = date.split('  ')[1]
    url = urls[n]['href']
    next_event_data = next_event_data.append({'event_title': ti, 'event_url': url, 'event_date': date}, ignore_index=True)
    n += 1

next_event_data



,event_title,event_url,event_date
0,UFC Fight Night: Rodriguez vs. Lemos,http://www.ufcstats.com/event-details/756f4590...,"November 05, 2022"
1,UFC 281: Adesanya vs. Pereira,http://www.ufcstats.com/event-details/b3b6e80b...,"November 12, 2022"
2,UFC Fight Night: Lewis vs. Spivac,http://www.ufcstats.com/event-details/012fc7cd...,"November 19, 2022"
3,UFC Fight Night: Thompson vs. Holland,http://www.ufcstats.com/event-details/b23388ff...,"December 03, 2022"
4,UFC 282: Prochazka vs. Teixeira 2,http://www.ufcstats.com/event-details/f65a0eb9...,"December 10, 2022"
5,UFC Fight Night: Cannonier vs. Strickland,http://www.ufcstats.com/event-details/56ec5895...,"December 17, 2022"
6,UFC 283,http://www.ufcstats.com/event-details/5717efc6...,"January 21, 2023"
7,UFC 284,http://www.ufcstats.com/event-details/01dd4cdc...,"February 11, 2023"


In [12]:
# load point
fights = pd.read_csv('data/ufc_stats/agg2/all_fights_9_27_V9.csv')
all_fights_no_drops = pd.read_csv('data/final/all_fights_no_drops_V6.csv')

In [41]:
pick_fight_num = 2

In [42]:
fighter_urls = get_fighter_urls(next_event_data['event_url'].values[pick_fight_num])
fighter_urls

,fighter1,fighter2:,fighter1_url,fighter2_url,fight#
0,Derrick Lewis,Serghei Spivac,http://www.ufcstats.com/fighter-details/d3df1a...,http://www.ufcstats.com/fighter-details/e2f6b2...,1
1,Cody Brundage,Rodolfo Vieira,http://www.ufcstats.com/fighter-details/f14f64...,http://www.ufcstats.com/fighter-details/260e9e...,2
2,Andre Fialho,Muslim Salikhov,http://www.ufcstats.com/fighter-details/33efaf...,http://www.ufcstats.com/fighter-details/447f98...,3
3,Charles Johnson,Zhalgas Zhumagulov,http://www.ufcstats.com/fighter-details/814e52...,http://www.ufcstats.com/fighter-details/55fa6c...,4
4,Jennifer Maia,Maryna Moroz,http://www.ufcstats.com/fighter-details/8e1f36...,http://www.ufcstats.com/fighter-details/b4192a...,5
5,Vince Morales,Jose Johnson,http://www.ufcstats.com/fighter-details/37098a...,http://www.ufcstats.com/fighter-details/84f727...,6
6,Vanessa Demopoulos,Maria Oliveira,http://www.ufcstats.com/fighter-details/2c483c...,http://www.ufcstats.com/fighter-details/37d752...,7


In [43]:
# turn event_date to actual date
next_event_data['event_date'] = pd.to_datetime(next_event_data['event_date']).dt.date
event_date = next_event_data['event_date'].values[pick_fight_num]
event_date

datetime.date(2022, 11, 19)

In [44]:
next_fight_df = fighter_urls[['fighter1', 'fighter2:']]
next_fight_df.columns = ['Fighter_A', 'Fighter_B']
next_fight_df['date'] = event_date
next_fight_df

,Fighter_A,Fighter_B,date
0,Derrick Lewis,Serghei Spivac,2022-11-19
1,Cody Brundage,Rodolfo Vieira,2022-11-19
2,Andre Fialho,Muslim Salikhov,2022-11-19
3,Charles Johnson,Zhalgas Zhumagulov,2022-11-19
4,Jennifer Maia,Maryna Moroz,2022-11-19
5,Vince Morales,Jose Johnson,2022-11-19
6,Vanessa Demopoulos,Maria Oliveira,2022-11-19


In [45]:
# replace all NANs with calculations
# First, ROLLING data
in_fight_cols = [n for n in list(all_fights_no_drops.columns) if n.startswith('A_') or n.startswith('B_')]
rolling_cols = [n for n in in_fight_cols if 'Rolling' in n]
in_fight_only_cols = [n for n in in_fight_cols if n not in rolling_cols]
in_fight_only_cols.remove('A_Height')
in_fight_only_cols.remove('B_Height')
in_fight_only_cols.remove('A_Reach')
in_fight_only_cols.remove('B_Reach')
in_fight_only_cols.remove('A_Leg_Reach')
in_fight_only_cols.remove('B_Leg_Reach')


In [46]:
A_cols = [n for n in in_fight_only_cols if n.startswith('A_')]
A_cols2 = pd.DataFrame(A_cols)
A_cols2['nonspecific'] = A_cols2[0].str[2:]
the_cols = list(A_cols2['nonspecific'].unique())

In [47]:
# for each column in all_metric_cols, get the mean, std, etc. for each fighter
def get_em(fighter, date, col_to_get, stat_to_calc):
    data = all_fights_no_drops[(all_fights_no_drops['Fighter_A'] == fighter) | (all_fights_no_drops['Fighter_B'] == fighter)]
    # only get fights before the date
    datey = pd.to_datetime(date)
    data['date'] = pd.to_datetime(data['date'])
    data = data[data['date'] < datey]
    # fighter could be either fighter A or fighter B
    fighter_data = pd.DataFrame()
    # when fighter is fighter A, get all fighter A data and copy it to fighter_data
    # fighterA df
    fighterA_df = all_fights_no_drops[all_fights_no_drops['Fighter_A'] == fighter]
    fighterB_df = all_fights_no_drops[all_fights_no_drops['Fighter_B'] == fighter]
    # keep only the fighters columns date, FighterA, and the col_to_get, do same for B, change col names from B to A, and concat
    fighterA_df = fighterA_df[['date', 'Fighter_A', 'A_' + col_to_get]]
    fighterA_df.rename(columns={'A_' + col_to_get: col_to_get, 'Fighter_A': 'fighter'}, inplace=True)
    fighterB_df = fighterB_df[['date', 'Fighter_B', 'B_' + col_to_get]]
    fighterB_df.rename(columns={'B_' + col_to_get: col_to_get, 'Fighter_B': 'fighter'}, inplace=True)
    fighter_data = fighter_data.append(fighterA_df)
    fighter_data = fighter_data.append(fighterB_df)
    # append the dataframes on fighter
    fighter_data = fighter_data.append(fighterA_df)
    fighter_data = fighter_data.append(fighterB_df)
    # make sure the date is before the date of the fight
    # get the average
    if stat_to_calc == 'mean':
        x = fighter_data[col_to_get].mean()
    elif stat_to_calc == 'std':
        x = fighter_data[col_to_get].std()
    elif stat_to_calc == 'max':
        x = fighter_data[col_to_get].max()
    elif stat_to_calc == 'min':
        x = fighter_data[col_to_get].min()
    elif stat_to_calc == 'median':
        x = fighter_data[col_to_get].median()  
    return x

In [48]:
next_event_date = next_fight_df['date'].values[0]
str(next_event_date)

'2022-11-19'

In [49]:
for col in the_cols:
    for stat in ['mean', 'std', 'max', 'min', 'median']:
        next_fight_df['A_Rolling_' + col + '_' + stat] = next_fight_df.apply(lambda x: get_em(fighter=x['Fighter_A'], date=next_event_date, col_to_get=col, stat_to_calc=stat), axis=1)
        next_fight_df['B_Rolling_' + col + '_' + stat] = next_fight_df.apply(lambda x: get_em(fighter=x['Fighter_B'], date=next_event_date, col_to_get=col, stat_to_calc=stat), axis=1)


In [50]:
next_fight_df.to_csv('data/final/next_fights/' + str(next_event_date) + '.csv')

In [51]:
next_fight_df.head(3)

,Fighter_A,Fighter_B,date,A_Rolling_Kd_mean,B_Rolling_Kd_mean,A_Rolling_Kd_std,B_Rolling_Kd_std,A_Rolling_Kd_max,B_Rolling_Kd_max,A_Rolling_Kd_min,...,A_Rolling_Ground_Strikes_percent_mean,B_Rolling_Ground_Strikes_percent_mean,A_Rolling_Ground_Strikes_percent_std,B_Rolling_Ground_Strikes_percent_std,A_Rolling_Ground_Strikes_percent_max,B_Rolling_Ground_Strikes_percent_max,A_Rolling_Ground_Strikes_percent_min,B_Rolling_Ground_Strikes_percent_min,A_Rolling_Ground_Strikes_percent_median,B_Rolling_Ground_Strikes_percent_median
0,Derrick Lewis,Serghei Spivac,2022-11-19,0.240,0.111111,0.473969,0.318728,2,1,0,...,0.673173,0.687851,0.281022,0.254282,1.0,1.0,0.000000,0.0,0.741935,0.705882
1,Cody Brundage,Rodolfo Vieira,2022-11-19,0.125,0.000000,0.341565,0.000000,1,0,0,...,0.702339,0.626623,0.267439,0.357207,1.0,1.0,0.333333,0.0,0.716667,0.629870
2,Andre Fialho,Muslim Salikhov,2022-11-19,0.375,0.333333,0.500000,0.481543,1,1,0,...,0.666667,0.600125,0.444444,0.294576,1.0,1.0,0.000000,0.0,1.000000,0.666667


#### Impute NAN with median

In [52]:
# impute NANs with median values - stdev

next_fight_df.fillna(all_fights_no_drops.median(), inplace=True)

In [53]:
next_fight_df.to_csv('data/final/next_fights/' + str(next_event_date) + '_imputed.csv')

In [54]:
next_fight_df

,Fighter_A,Fighter_B,date,A_Rolling_Kd_mean,B_Rolling_Kd_mean,A_Rolling_Kd_std,B_Rolling_Kd_std,A_Rolling_Kd_max,B_Rolling_Kd_max,A_Rolling_Kd_min,...,A_Rolling_Ground_Strikes_percent_mean,B_Rolling_Ground_Strikes_percent_mean,A_Rolling_Ground_Strikes_percent_std,B_Rolling_Ground_Strikes_percent_std,A_Rolling_Ground_Strikes_percent_max,B_Rolling_Ground_Strikes_percent_max,A_Rolling_Ground_Strikes_percent_min,B_Rolling_Ground_Strikes_percent_min,A_Rolling_Ground_Strikes_percent_median,B_Rolling_Ground_Strikes_percent_median
0,Derrick Lewis,Serghei Spivac,2022-11-19,0.2400,0.111111,0.473969,0.318728,2,1,0,...,0.673173,0.687851,0.281022,0.254282,1.0,1.000000,0.000000,0.00,0.741935,0.705882
1,Cody Brundage,Rodolfo Vieira,2022-11-19,0.1250,0.000000,0.341565,0.000000,1,0,0,...,0.702339,0.626623,0.267439,0.357207,1.0,1.000000,0.333333,0.00,0.716667,0.629870
2,Andre Fialho,Muslim Salikhov,2022-11-19,0.3750,0.333333,0.500000,0.481543,1,1,0,...,0.666667,0.600125,0.444444,0.294576,1.0,1.000000,0.000000,0.00,1.000000,0.666667
3,Charles Johnson,Zhalgas Zhumagulov,2022-11-19,0.0000,0.200000,0.000000,0.615587,0,2,0,...,1.000000,0.541667,0.000000,0.385758,1.0,1.000000,1.000000,0.00,1.000000,0.583333
4,Jennifer Maia,Maryna Moroz,2022-11-19,0.0000,0.062500,0.000000,0.245935,0,1,0,...,0.775758,0.725493,0.342830,0.312413,1.0,1.000000,0.000000,0.00,1.000000,0.801587
5,Vince Morales,Jose Johnson,2022-11-19,0.3125,0.000000,0.592289,0.000000,2,0,0,...,0.696296,0.595000,0.344782,0.178979,1.0,0.750000,0.000000,0.44,0.750000,0.595000
6,Vanessa Demopoulos,Maria Oliveira,2022-11-19,0.0000,0.000000,0.000000,0.000000,0,0,0,...,0.803268,0.571023,0.154495,0.391993,1.0,0.909091,0.676471,0.00,0.733333,0.687500
